<a href="https://colab.research.google.com/github/pikey-msc/RiesgosFinancieros/blob/master/2025-1/Swaps_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Cargar info de github

In [1]:
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"

Cloning into 'RiesgosFinancieros'...
remote: Enumerating objects: 1108, done.
remote: Counting objects: 100% (340/340), done.
remote: Compressing objects: 100% (164/164), done.
remote: Total 1108 (delta 189), reused 279 (delta 158), pack-reused 768 (from 1)
Receiving objects: 100% (1108/1108), 63.88 MiB | 9.02 MiB/s, done.
Resolving deltas: 100% (694/694), done.
Updating files: 100% (436/436), done.


## Carga información insumos

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

fval = datetime.strptime("20230310", "%Y%m%d") #Fecha de valoración
itpl = 0 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada
alpha = 0.98 #Nivel de confianza para obtener estimaciones de riesgo

#SWAP
btasadesc_sw="RiesgosFinancieros/2024-1/Tarea/tasa_TIIE_SW_OP.txt"
btasacupvar_sw="RiesgosFinancieros/2024-1/Tarea/tasa_DIRS_SW_OP.txt"
tasafija_sw=np.array([0.079,0.075]) #se establece la tasa fija a pagar para cada swap
plazos_sw=np.array([588,360]) #se establece el número de días que vivirá el swap
plazocupon_sw=np.array([28,28]) #se establece el número de días que se pagará cada cupón
contratos_sw=np.array([-1600,1000]) #se establece el número de contratos_sw de cada swap
nominal_sw=np.array([1,1]) #se establece el nominal_sw de cada swap
por_sw=np.array([0,0]) #se establece el porcentaje de cada swap



data1 = pd.read_table(btasadesc_sw,header=None)
n1 = data1.shape[0]
m1_orig_sw = data1.shape[1]
X1_orig_sw = pd.DataFrame(data1.iloc[1:n1, 1:m1_orig_sw], dtype=float)
X1_orig_sw['Date'] = pd.to_datetime(data1.iloc[1:n1, 0], format = '%Y%m%d')
nodos1_sw = data1.iloc[0, 1:m1_orig_sw]

data2 = pd.read_table(btasacupvar_sw,header=None)
n2 = data2.shape[0]
m2_orig_sw = data2.shape[1]
X2_orig_sw = pd.DataFrame(data2.iloc[1:n2, 1:m2_orig_sw], dtype=float)
X2_orig_sw['Date'] = pd.to_datetime(data2.iloc[1:n1, 0], format = '%Y%m%d')
nodos2_sw = data2.iloc[0, 1:m2_orig_sw]


## Emparejar por fecha los insumos

In [3]:
#función para unir por fecha
def filter_on_date(base_df, join_df, description):
    # Asegurarse de que ambos objetos sean DataFrames con una columna 'Date'
    if not isinstance(base_df, pd.DataFrame):
        base_df = pd.DataFrame({'Date': base_df})
    if not isinstance(join_df, pd.DataFrame):
        join_df = pd.DataFrame({'Date': join_df})

    result = pd.merge(base_df, join_df, on='Date', how='inner')[['Date']].drop_duplicates()
    print(f"Number of rows in {description}: {len(result)}")
    return result

# poner los conjuntos que se unirán quitando el primero
dataframes_to_join = [(X2_orig_sw, "Swapsvp_swaps_cupon"),]

#conjunto primero a unir por fecha
current_df = X1_orig_sw

# ciclo para unir conjuntos por fecha
for join_df, description in dataframes_to_join:
    current_df = filter_on_date(current_df, join_df, description)

# Mostrar el DataFrame final
print(current_df.drop_duplicates())

# Suponemos que lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol y otras DataFrames ya están definidas

# Calcula el número de filas
n = len(current_df)

# ordenar data frame
current_df=current_df.sort_values(by="Date", ascending=False)

# Borrar las fechas ya alineadas y ordenadas y quedarse con las tasas
def join_and_drop_date(base_df, join_df, divide_by_100=False):
    result = pd.merge(base_df, join_df, on='Date', how='inner').sort_values('Date', ascending=False)
    result.drop(columns=['Date'], inplace=True)
    if divide_by_100:
        result = result / 100
    return result

# Historia de curvas de bonde
X1_orig_sw = join_and_drop_date(current_df, X1_orig_sw)
X2_orig_sw = join_and_drop_date(current_df, X2_orig_sw)


Number of rows in Swapsvp_swaps_cupon: 255
          Date
0   2023-09-29
1   2023-09-28
2   2023-09-27
3   2023-09-26
4   2023-09-25
..         ...
250 2022-10-04
251 2022-10-03
252 2022-09-30
253 2022-09-29
254 2022-09-28

[255 rows x 1 columns]


# Enfoque a portafolio de insumos

In [4]:
def approx(x, y, new_x, rule=2):
    return np.interp(new_x, x, y.astype(float))


nodosvp=nodos1_sw
nodostc=nodos2_sw
curvavp=X1_orig_sw
curvatc=X2_orig_sw
n1=len(curvavp)
n2=len(curvatc)


m = len(plazos_sw)
N = (plazos_sw // plazocupon_sw) + 1  # número de cupones a pagar

# Inicialización de matrices/vectores
VTplazos_sw = np.zeros(np.sum(N))
VTplazos_swc = np.zeros(np.sum(N))
contratos_swT = np.zeros(np.sum(N))
por_swT=np.zeros(sum(N)) #vector de todos los dummy si paga o recibe de todos los flujos de todos los contratos_sw
plazocupon_swT = np.zeros(np.sum(N))
tasafija_swT = np.zeros(np.sum(N))
nominal_swT = np.zeros(np.sum(N))
Xvp = np.zeros((n, len(VTplazos_sw)))
Xtc = np.zeros((n, len(VTplazos_sw)))
Xtcc = np.zeros((n, len(VTplazos_sw)))
XtfwdT = np.zeros((n, len(VTplazos_sw)))

# Cálculos adicionales
plazini_sw = plazos_sw - plazocupon_sw * (N - 1)

from scipy.interpolate import interp1d

for j in range(m):
    sum_N = np.sum(N[:j+1])
    sum_N_prev = np.sum(N[:j]) if j > 0 else 0

    if j == 0:
        VTplazos_sw[:sum_N] = np.arange(plazini_sw[j], plazos_sw[j] + 1, plazocupon_sw[j])
        VTplazos_swc[:sum_N] = np.concatenate(([0], VTplazos_sw[0:(sum_N - 1)]))
        contratos_swT[:sum_N] = contratos_sw[j]
        plazocupon_swT[:sum_N] = plazocupon_sw[j]
        nominal_swT[:sum_N] = nominal_sw[j]
        por_swT[:sum_N] = por_sw[j]
        tasafija_swT[:sum_N] = tasafija_sw[j]
    else:
        VTplazos_sw[sum_N_prev:sum_N] = np.arange(plazini_sw[j], plazos_sw[j] + 1, plazocupon_sw[j])
        VTplazos_swc[sum_N_prev:sum_N] = np.concatenate(([0], VTplazos_sw[sum_N_prev:(sum_N - 1)]))
        contratos_swT[sum_N_prev:sum_N] = contratos_sw[j]
        plazocupon_swT[sum_N_prev:sum_N] = plazocupon_sw[j]
        nominal_swT[sum_N_prev:sum_N] = nominal_sw[j]
        por_swT[sum_N_prev:sum_N] = por_sw[j]
        tasafija_swT[sum_N_prev:sum_N] = tasafija_sw[j]


for i in range(n):
    if itpl == 0:
        Xvp[i, :] = approx(nodosvp, curvavp.iloc[i, :], VTplazos_sw)
        Xtc[i, :] = approx(nodostc, curvatc.iloc[i, :], VTplazos_sw)
        Xtcc[i, :] = approx(nodostc, curvatc.iloc[i, :], VTplazos_swc)
    else:
        Xvp[i, :] = talamb(nodosvp, curvavp.iloc[i, :], VTplazos_sw)
        Xtc[i, :] = talamb(nodostc, curvatc.iloc[i, :], VTplazos_sw)
        Xtcc[i, :] = talamb(nodostc, curvatc.iloc[i, :], VTplazos_swc)

    XtfwdT[i, :] = ((1 + Xtc[i, :] * VTplazos_sw / 360) / (1 + Xtcc[i, :] * VTplazos_swc / 360) - 1) * 360 / plazocupon_swT

    j = 0
    while j < len(VTplazos_sw):
        if VTplazos_sw[j] <= plazocupon_swT[j]:
            XtfwdT[i, j] = Xtc[i, j]
        else:
            j = np.sum(N[0:(j+1)]) - 1
        j += 1

X_sw = np.hstack((XtfwdT, Xvp))

# Valuación

**Características**

Valor nominal: Generalmente, 100 pesos pero varía por emisora.

Plazo: de 1 a 360 días, según las necesidades de financiamiento de la empresa emisora.

Rendimiento: al igual que los bonos, este instrumento se compra a descuento respecto de su valor nominal, pero por lo general pagan una sobretasa referenciada a tasa fija o variable.

Garantía: este título, por ser un pagaré, no ofrece ninguna garantía, por lo que es importante evaluar bien al emisor.



$$\Large V=\sum_{i=1}^{n}\frac{N\cdot C \cdot t_{c_{p_i}}\cdot p_c/360}{(1+t_{vp_{p_i}} \cdot p_i/360)} + \frac{N\cdot C}{(1+t_{vp_{p_n}} \cdot p_n/360)}$$
	Donde: \\
$N$: Valor Nominal del bono\
$C$: Número de contratos\
$p_{c}$: Plazo fijo para cada pago de intereses del cupón.\
$p_{i}$: Plazo acumulado (en días) al cupón $i$.\
$t_{c_{p_i}}$: Tasa cupón variable, se obtiene de la curva subyacente que le corresponda, casi siempre con la tasa forward entre $p_{i-1}$ y $p_i$.\
$t_{vp_{p_i}}$: Tasa valor presente que depende de la curva de bonos según el plazo acumulado al pago del cupón $i$.


En general tenemos dos factores de riesgo subyacente (la curva de valor presente y la curva de cupones) pero como cada cupón tiene ``$n$'' flujos entonces tiene $n$ factores de riesgo para los valores presentes y $n$ factores de riesgo para los cupones, por lo que tienen $2n$ factores de riesgo específicos que provienen de dos factores de riesgo subyacentes.
Para el caso de $K$ bonos cupón variable el número de factores de riesgo sería $2\sum_{i=1}^{K}n_i$, donde $n_i$ es el número de cupones a pagar del bono $i$.

In [5]:

def swap(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp, N):
    V0 = np.zeros(len(N))
    V0f = (contratos_swT * (XtfwdT - tasafija_swT) * (plazocupon_swT / 360)) / ((1 + Xvp * VTplazos_sw / 360) * nominal_swT * ((-1) ** por_swT))

    for j in range(len(N)):
        if j == 0:
            V0[j] = np.sum(V0f[0:N[j]])
        else:
            V0[j] = np.sum(V0f[np.sum(N[0:j]):np.sum(N[0:(j+1)])])

    return V0

V0_sw = swap(por_swT, contratos_swT, nominal_swT, XtfwdT[0, :], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[0, :], N)
print(V0_sw)


[-2424.31271961   869.80812837]
